# Manipulation de fichier

SparkSQL est capable de lire et écrire des données depuis ou vers des fichiers de différents formats. Ces fichiers peuvent être des fichiers simples, des fichiers compressés, des fichiers partitionnés, des fichiers partitionnés sur HDFS.

Dans ce notebook, nous allons voir comment se comporte SparkSQL avec les fichiers.

## Préambule

In [ ]:
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`
import $ivy.`org.apache.logging.log4j:log4j-api:2.8.2`
import $ivy.`org.apache.logging.log4j:log4j-slf4j-impl:2.8.2`

// Avoid disturbing logs
import org.apache.log4j._
import org.apache.log4j.varia._
BasicConfigurator.configure(NullAppender.getNullAppender())

import $ivy.`org.apache.spark::spark-core:3.2.1`
import $ivy.`org.apache.spark::spark-sql:3.2.1`
import $ivy.`org.apache.spark::spark-hive:3.2.1`

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .appName("Spark tunning - Fichiers")
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
    .config("spark.sql.warehouse.dir", "target/spark-warehouse")
    // .config("spark.hadoop.hive.metastore.warehouse.dir", "target/metastore")
    // .enableHiveSupport()
    .getOrCreate()
}

import spark.implicits._

Exécutez la cellule ci-dessous à chaque fois que vous souhaitez recommencer les exercices plus bas.

In [ ]:
def cleanTarget = {
    import java.nio.file.{Files, Path, Paths}

    def deleteDirectory(path: Path): Unit = {
      if (Files.isDirectory(path)) {
        // List the directory contents and delete them recursively
        Files.list(path).forEach(deleteDirectory)
      }
      // Delete the file or directory (if it's a directory, it should be empty by now)
      try { Files.delete(path) } catch { case _: java.nio.file.NoSuchFileException => () }
    }

    val targetDirectory = Paths.get("target/")
    deleteDirectory(targetDirectory)
    Files.createDirectory(targetDirectory)
}

cleanTarget

## Lecture d'un fichier JSON compressé

SparkSQL est capable de gérer naturellement les fichiers compressés, sur différents algorithmes de compression (gzip, bzip2, snappy, lz4, zstd...). La compression permet de gagner de l'espace de stockage et d'augmenter le débit du transfert de données. Il sera en général plus efficace sur les fichiers textes que sur les fichiers binaires. La compression demande un peu plus d'utilisation CPU.

Utilisez Spark pour charger le fichier JSON compressé.

La méthode `.repartition()` plus bas permet de forcer la redistribution des données dans plusieurs partitions. La valeur passée en paramètre correspond au nombre de partitions souhaité. Cette valeur est limitée par le nombre de Core/CPU disponibles.

In [ ]:
val rawDataframe =
  spark.read
    .json("data/tweets.json.gz")
    .cache()
    .where($"_corrupt_record" isNull)
    .drop("_corrupt_record")

val dataframe =
  rawDataframe
    .repartition(4)

dataframe.show(numRows = 10)

Nous allons voir combien de partitions sont associées au dataframe. Pour cela, nous allons utiliser l'interface RDD.

In [ ]:
dataframe.rdd.getNumPartitions

## Sauvegarde dans des fichiers Parquet

Nous allons maintenant tester différents algorithmes de compression.

La fonction ci-dessous va permettre de visualiser pour chaque algorithme ses performances et termes de capacité de compression.

In [ ]:
def testSaveParquet(dataframe: DataFrame, alg: String): Unit = {
  val file = s"tweets-$alg.parquet"
  dataframe.repartition(8).write.option("compression", alg).parquet(s"target/$file")
}

👷 Dans chaque cas ci-dessous, regardez et comparez les différents résultats obtenus, aussi bien dans Spark UI qu'au niveau du système de fichiers (à gauche de l'écran).

### Pas de compression

In [ ]:
testSaveParquet(dataframe, "none")

### Snappy compression

In [ ]:
testSaveParquet(dataframe, "snappy")

### GZip

In [ ]:
testSaveParquet(dataframe, "gzip")

### ZStd

In [ ]:
testSaveParquet(dataframe, "zstd")

In [ ]:
val tweets = spark.read.parquet("target/tweets-gzip.parquet")

tweets.show(numRows = 5)

In [ ]:
tweets.rdd.getNumPartitions

## Gestion par Spark

En utilisant les méthodes `.saveAsTable()` et `table()`, Spark va gérer sont propre espace de stockage. Ici, il s'agit de `target/spark-warehouse` (par défaut, `spark-warehouse`).

👷 Exécuter les cellules ci-dessous.

In [ ]:
dataframe.write.mode("overwrite").saveAsTable("tweets")

In [ ]:
spark.read.table("tweets").show()

🤔 **Question** 🤔

* Explorez le répertoire `target/spark-warehouse`. Comment est-ce que Spark organise le stockage et quel fomrat est utilisé ?
* Quelles différences apparaissent avec l'approche vu précédemment ?